In [1]:
from uuid import uuid1
import random

In [2]:
class Car:
    def __init__(self):
        self.id = str(uuid1())
        self.type = "Car"

class Person:
    def __init__(self):
        self.id = str(uuid1())
        self.type = "Person"
        self.cars = []
        
    def add_car(self, car):
        self.cars.append(car)

In [3]:
cars = [Car() for x in range(0, 1000)]

In [4]:
persons = [Person() for x in range(0, 100)]

In [5]:
for person in persons:
    [person.add_car(random.choice(cars)) for x in range(0,20)]

persons[0].cars

In [6]:
from py2neo import Graph, Path, Node, Relationship, NodeMatcher
graph = Graph()
matcher = NodeMatcher(graph)

# clear db
graph.evaluate("MATCH (n) DETACH DELETE n")

In [7]:
matcher = NodeMatcher(graph)

for car in cars:
    tx = graph.begin()
    node = Node('Car', id=car.id)
    tx.create(node)
    tx.commit()
    
for person in persons:
    tx = graph.begin()
    p = Node('Person', id=person.id)
    tx.create(p)
    
    for car in person.cars:
        car_node = matcher.match('Car', id=car.id).first()
        rel = Relationship(p, 'OWNED', car_node)
        tx.create(rel)
    tx.commit()

In [10]:
from pymongo import MongoClient

mongo = MongoClient()
mongodb = mongo['test-db']
mongocol = mongodb['ownership']

In [11]:
for person in persons:
    p_node = {
        'id': person.id,
        'cars': [car.id for car in person.cars]
    }
    
    mongocol.insert_one(p_node)

In [18]:
import timeit
def query_neo4j():
    car = random.choice(cars)
    graph.evaluate("MATCH (c:Car) <-[:OWNED]- (p:Person) -[:OWNED]-> (:Car {id:'" + car.id + "'}) return p,c")
    
timeit.timeit(stmt=query_neo4j, number=20)

db65deb4-6d51-11e8-a89a-9cb6d0d3a297
db633fa6-6d51-11e8-a89a-9cb6d0d3a297
db635a0e-6d51-11e8-a89a-9cb6d0d3a297
db65109c-6d51-11e8-a89a-9cb6d0d3a297
db62bd60-6d51-11e8-a89a-9cb6d0d3a297
db65bd58-6d51-11e8-a89a-9cb6d0d3a297
db62b18a-6d51-11e8-a89a-9cb6d0d3a297
db62f924-6d51-11e8-a89a-9cb6d0d3a297
db64c63c-6d51-11e8-a89a-9cb6d0d3a297
db62f924-6d51-11e8-a89a-9cb6d0d3a297
db65d766-6d51-11e8-a89a-9cb6d0d3a297
db654f62-6d51-11e8-a89a-9cb6d0d3a297
db65433c-6d51-11e8-a89a-9cb6d0d3a297
db63979e-6d51-11e8-a89a-9cb6d0d3a297
db63ef32-6d51-11e8-a89a-9cb6d0d3a297
db64100c-6d51-11e8-a89a-9cb6d0d3a297
db65c938-6d51-11e8-a89a-9cb6d0d3a297
db656eca-6d51-11e8-a89a-9cb6d0d3a297
db635cf2-6d51-11e8-a89a-9cb6d0d3a297
db64fb48-6d51-11e8-a89a-9cb6d0d3a297


0.44601931900251657

In [29]:
def query_mongo():
    car = random.choice(cars)
    mongocol.find({"cars":car.id}).count()
    
timeit.timeit(stmt=query_mongo, number=20)

2
2
1
0
0
2
3
3
2
1
2
2
3
0
4
3
5
5
1
2


0.05064530100207776